# Inferring prompt types

This notebook demos two transformers, which broadly aim at producing abstract representations of an utterance in terms of its phrasing and its rhetorical intent: 

* The `PhrasingMotifs` transformer extracts representations of utterances in terms of how they are phrased;
* The `PromptTypes` transformer computes latent representations of utterances in terms of their rhetorical intention -- the _responses_ they aim at prompting -- and assigns utterances to different (automatically-inferred) types of intentions.

It also demos some additional transformers used in preprocessing steps.



Together, these transformers implement the methodology detailed in the [paper](http://www.cs.cornell.edu/~cristian/Asking_too_much.html), 

```
Asking Too Much? The Rhetorical Role of Questions in Political Discourse 
Justine Zhang, Arthur Spirling, Cristian Danescu-Niculescu-Mizil
Proceedings of EMNLP 2017
```

ConvoKit also includes an end-to-end implementation, `PromptTypesWrapper`, that runs the transformers one after another, and handles the particular pre-processing steps found in the paper. See [this notebook](https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit/blob/master/examples/prompt-types/prompt-type-wrapper-demo.ipynb) for a demonstration of this end-to-end transformer.

This is a really clear example of a method which reflects both good (we think) ideas and somewhat ad-hoc implementation decisions. As such, there are lots of options and potential variations to consider (beyond the deeper question of what phrasings and intentions even are) -- I'll detail these as I go along.

Note that due to small methodological tweaks and changes in the random seed, the particular output of the transformers as presently implemented may not totally match the output from the paper, but the broad types of questions returned are comparable.

## Preliminaries

First we load the corpus. We will examine a dataset of questions from question periods that take place in the British House of Commons (also detailed in the paper). 

In [1]:
import os

In [2]:
os.chdir('../..')

In [3]:
import convokit

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
from convokit import download

We'll load the corpus, plus some pre-computed dependency parses (see [this notebook](https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit/blob/master/examples/text-processing/text_preprocessing_demo.ipynb) for a demonstration of how to get these parses on your own; for this dataset they should be included with our release).

In [6]:
# OPTION 1: DOWNLOAD CORPUS 
# UNCOMMENT THESE LINES TO DOWNLOAD CORPUS
DATA_DIR = './'
ROOT_DIR = download('parliament-corpus')

# OPTION 2: READ PREVIOUSLY-DOWNLOADED CORPUS FROM DISK
# UNCOMMENT THIS LINE AND REPLACE WITH THE DIRECTORY WHERE THE PARLIAMENT-CORPUS IS LOCATED
# ROOT_DIR = '<YOUR DIRECTORY>'

corpus = convokit.Corpus(ROOT_DIR)
corpus.load_info('utterance',['parsed'])

Dataset already exists at /Users/calebchiam/Documents/test_env/parliament-corpus


In [7]:
VERBOSITY = 10000

Our specific goal, which we'll use ConvoKit to accomplish, is to produce an abstract representation of questions asked by members of parliament, in terms of:

* how they are phrased: what phrasing, or lexico-syntatic "motif", does a question have? 
* their rhetorical intention: what's the intent of the asker -- which we take to mean the response the asker aims to prompt? 

In other words, what are the different types of questions people ask in parliament?

Here's an example of an utterance:

In [8]:
test_utt_id = '1997-01-27a.4.0'
utt = corpus.get_utterance(test_utt_id)

In [9]:
utt.text

"Does my right hon Friend agree that last week 's statement about a replacement royal yacht has been widely welcomed ? Does he agree also that , ideally , Britannia should become the centrepiece of the millennium project in Portsmouth harbour , spanning Gosport and Portsmouth ? I am sure that that idea would prove very popular . As to plans for a new yacht , does my right hon Friend share my distaste for the Opposition 's tactics ? They had every opportunity to express their grudging and negative attitude during the past two years when the project was under discussion ."

To state our goals more precisely:

* For each _sentence_ that has a question (all but the last), we want to come up with a representation of the sentence's phrasing. Intuitively, for instance, the first two sentences sound like they could both be thought of as a "Does X agree that Y?" -- whether Y is asking about a yacht or a harbour. 
* For each utterance, we want to come up with a representation of the utterance's rhetorical intent. Intuitively, all the questions could be construed as asking if the answerer is in agreement with the asker -- whether they "agree" with the opinion or "share" the opinion. We might think of this as being an example of an "agreeing" type of question.

Intuitively, if we want to get at this higher level of abstraction, we have to look beyond the particular n-grams: it doesn't seem plausible that there is a meaningful type of question about yachts (unless our specific context is the parliamentary subcommittee on yachts). 



## Preprocessing step: Arcs

One place to start is to look at the structural "skeleton" of sentences -- i.e., its dependency parse. Thus, we are first going to provide a representation of questions in terms of their dependency parse by extracting all the parent-to-child token edges, or "arcs". We will use the `TextToArcs` class to do this:

In [10]:
from convokit.text_processing import TextToArcs

`get_arcs` is a transformer (actually a `TextProcessor`) that will read the dependency parse of an utterance and write the resultant arcs to a field called `'arcs'`:

In [11]:
get_arcs = TextToArcs('arcs', verbosity=VERBOSITY)
corpus = get_arcs.transform(corpus)

10000/433787 utterances processed
20000/433787 utterances processed
30000/433787 utterances processed
40000/433787 utterances processed
50000/433787 utterances processed
60000/433787 utterances processed
70000/433787 utterances processed
80000/433787 utterances processed
90000/433787 utterances processed
100000/433787 utterances processed
110000/433787 utterances processed
120000/433787 utterances processed
130000/433787 utterances processed
140000/433787 utterances processed
150000/433787 utterances processed
160000/433787 utterances processed
170000/433787 utterances processed
180000/433787 utterances processed
190000/433787 utterances processed
200000/433787 utterances processed
210000/433787 utterances processed
220000/433787 utterances processed
230000/433787 utterances processed
240000/433787 utterances processed
250000/433787 utterances processed
260000/433787 utterances processed
270000/433787 utterances processed
280000/433787 utterances processed
290000/433787 utterances proc

`'arcs'` is a list where each element corresponds to a sentence in the utterance. Each sentence is represented in terms of its arcs, in a space-separated string. 

Each arc, in turn, can be read as follows:

* `x_y` means that `x` is the parent and `y` is the child token (e.g., `agree_does` = `agree --> does`)
* `x_*` means that `x` is a token with at least one descendant, which we do not resolve (this is roughly like bigrams backing off to unigrams)
* `x>y` means that `x` and `y` are the first two tokens in the sentence (the decision here was that how the sentence starts is a signal of "phrasing structure" on par with the dependency tree structure)
* `x>*` means that `x` is the first token in the sentence. 

In [12]:
utt.get_info('arcs')

["'s_* a_* about_* about_yacht agree_* agree_does agree_hon agree_welcomed been_* does>* does>my does_* friend_* has_* hon_* hon_friend hon_my hon_right last_* my_* replacement_* right_* royal_* statement_* statement_about statement_week that_* week_'s week_* week_last welcomed_* welcomed_been welcomed_has welcomed_statement welcomed_that welcomed_widely widely_* yacht_* yacht_a yacht_replacement yacht_royal",
 'agree_* agree_also agree_become agree_does agree_he also_* become_* become_britannia become_centrepiece become_ideally become_should become_spanning become_that britannia_* centrepiece_* centrepiece_of centrepiece_the does>* does>he does_* gosport_* harbour_* harbour_portsmouth he_* ideally_* in_* in_harbour millennium_* of_* of_project portsmouth_* project_* project_in project_millennium project_the should_* spanning_* spanning_gosport that_* the_*',
 'am_* am_i am_sure i>* i_* idea_* idea_that popular_* popular_very prove_* prove_idea prove_popular prove_that prove_would sure

### Further preprocessing: cleaned-up arcs

At this point, while we've got the methodology to start making sense of the dependency tree, we arguably haven't progressed beyond producing fancy bigram representations of sentences. One problem is perhaps that the default arc extraction is a bit too permissive -- it gives us _all_ of the arcs. We might not want this for a few reasons:

* We only want to learn about question phrasings; we don't actually care about non-question sentences.
* The structure of a question might be best encapsulated by the arcs that go out of the _root_ of the tree; as you get further down we might end up with less structural and more content-specific representations.
* Likewise, the particular _nouns_ used (e.g., `yacht`) might not be good descriptions of the more abstract phrasing pattern.

All of these points are debatable, and the resultant modules I'll show below hopefully allow you to play around with them. Taking these point as is for now, though, we'll do the following.

In [13]:
from convokit.phrasing_motifs import CensorNouns, QuestionSentences
from convokit.convokitPipeline import ConvokitPipeline

We will actually create a pipeline to extract the arcs we want. This pipeline has the following components, in order:

* `CensorNouns`: a transformer that removes all the nouns and pronouns from a dependency parse. This transformer also collapses constructions like `What time [is it]` into `What [is it]`.
* `TextToArcs`: calling the arc extractor from above with an extra parameter: `root_only=True` which will only extract arcs attached to the root (in addition to the first two tokens, though this is also tunable by passing in parameter `use_start=True`).
* `QuestionSentences`: a transformer that, given utterance fields consisting of a list of sentences, removes all the sentences which contain question marks. Here, we pass an extra parameter `input_filter=question_filter`, telling it to ignore utterances which aren't listed in the Corpus as questions (i.e., if a player asks a question, we'll discount this, since it's not labeled in the Corpus as a reporter question). 
    * (you may wonder how this transformer can tell whether a sentence has a question mark in it, given that the output of `TextToArcs` doesn't have any punctuation. Under the hood, `QuestionSentences` looks at the dependency parse of the sentence and checks whether the last token is a question.)
    * `QuestionSentences` also omits any sentences which don't begin in capital letters. To turn this off, pass parameter `use_caps=False`.

In [14]:
def question_filter(utt, aux_input={}):
    return utt.meta['is_question']

In [15]:
q_arc_pipe = ConvokitPipeline([
    ('censor_nouns', CensorNouns('parsed_censored', verbosity=VERBOSITY)),
    ('shallow_arcs', TextToArcs('arcs_censored', input_field='parsed_censored', 
                               root_only=True, verbosity=VERBOSITY)),
    ('question_sentence_filter', QuestionSentences('question_arcs', input_field='arcs_censored',
                                         input_filter=question_filter, verbosity=VERBOSITY))
])

In [16]:
corpus = q_arc_pipe.transform(corpus)

10000/433787 utterances processed
20000/433787 utterances processed
30000/433787 utterances processed
40000/433787 utterances processed
50000/433787 utterances processed
60000/433787 utterances processed
70000/433787 utterances processed
80000/433787 utterances processed
90000/433787 utterances processed
100000/433787 utterances processed
110000/433787 utterances processed
120000/433787 utterances processed
130000/433787 utterances processed
140000/433787 utterances processed
150000/433787 utterances processed
160000/433787 utterances processed
170000/433787 utterances processed
180000/433787 utterances processed
190000/433787 utterances processed
200000/433787 utterances processed
210000/433787 utterances processed
220000/433787 utterances processed
230000/433787 utterances processed
240000/433787 utterances processed
250000/433787 utterances processed
260000/433787 utterances processed
270000/433787 utterances processed
280000/433787 utterances processed
290000/433787 utterances proc

This pipeline results in a more minimalistic representation of utterances, in terms of just the arcs at the root of dependency trees, just the questions, and no nouns:

In [17]:
utt.get_info('question_arcs')

['agree_* agree_does agree_welcomed does>*',
 'agree_* agree_also agree_become agree_does does>*',
 'as>* as>to share_* share_does']

Here's another example:

In [18]:
test_utt_id_1 = '2015-06-09c.1041.5'
utt1 = corpus.get_utterance(test_utt_id_1)

In [19]:
utt1.text

'Given what the Foreign Secretary has said about the importance of the Iran discussions on the nuclear agreement , what is he doing to ensure greater clarity about the baselines , the extent of the inspection regime and the consequences of infringement ? Given that the agreement will allow advanced centrifuge , the infringements might arrive a little earlier than anticipated .'

In [20]:
utt1.get_info('question_arcs')

['doing_* doing_ensure doing_given doing_is doing_what given>* given>what']

## Phrasing motifs

Finally, to arrive at our representation of phrasings, we can go one further level of abstraction. In short, some of these arcs feel less fully-specified than others. While `agree_does` sounds like it hints at a coherent question, `doing_is` seems like it's not meaningful until you consider that it occurs in the same sentence as `doing_ensure` (i.e., "_what is the Government doing to ensure...?_")

Our intuition is to think of phrasings as frequently-cooccurring sets of multiple arcs. To extract these frequent arc-sets (which may remind you of the data mining idea of extracting frequent itemsets) we will use the `PhrasingMotifs` class.

In [21]:
from convokit.phrasing_motifs import PhrasingMotifs

In [22]:
pm_model = PhrasingMotifs('motifs','question_arcs',min_support=100,fit_filter=question_filter,
                          verbosity=VERBOSITY)

Here, `pm_model` will:

* extract all sets of arcs, as read from the `question_arcs` field, which occur at least 50 times in a corpus. These frequently-occurring arc sets will constitute the set, or "vocabulary", of phrasings.
* write the resultant output -- the phrasings that an utterance contains -- to a field called `question_motifs`. 

On the latter point, `pm_model` will only transform (i.e., label phrasings for) utterances which are questions, i.e., `question_filter(utterance) = True`. That is, in both the train and transform steps, we totally ignore non-questions.

Note that the phrasings learned by `pm_model` are therefore _corpus-specific_ -- different corpora may have different frequently-occurring sets, resulting in different vocabularies of phrasings. For instance, you wouldn't expect people in the British House of Commons to ask questions that sound like questions asked to tennis players. In this respect, think of `PhrasingMotifs` like models from scikit learn (e.g., `LogisticRegression`) -- it is fit to a particular dataset:

In [48]:
pm_model.fit(corpus)

counting frequent itemsets for 325339 sets
	first pass: counting itemsets up to and including 5 items large
	first pass: 10000/325339 sets processed
	first pass: 20000/325339 sets processed
	first pass: 30000/325339 sets processed
	first pass: 40000/325339 sets processed
	first pass: 50000/325339 sets processed
	first pass: 60000/325339 sets processed
	first pass: 70000/325339 sets processed
	first pass: 80000/325339 sets processed
	first pass: 90000/325339 sets processed
	first pass: 100000/325339 sets processed
	first pass: 110000/325339 sets processed
	first pass: 120000/325339 sets processed
	first pass: 130000/325339 sets processed
	first pass: 140000/325339 sets processed
	first pass: 150000/325339 sets processed
	first pass: 160000/325339 sets processed
	first pass: 170000/325339 sets processed
	first pass: 180000/325339 sets processed
	first pass: 190000/325339 sets processed
	first pass: 200000/325339 sets processed
	first pass: 210000/325339 sets processed
	first pass: 220000

Here are the most common phrasings and how often they occur in the data (in # of sentences). Note that `('*',)` denotes the null phrasing -- i.e., it encapsulates sentences with _any_ root word. 

In [49]:
pm_model.print_top_phrasings(25)

('*',) 325339
('will>*',) 67920
('does>*',) 59959
('is_*',) 57904
('is>*',) 45238
('is>*', 'is_*') 42850
('agree_*',) 36085
('agree_does',) 33685
('agree_*', 'agree_does') 33685
('agree_*', 'does>*') 30009
('agree_does', 'does>*') 29984
('agree_*', 'agree_does', 'does>*') 29984
('is_aware',) 22049
('is_*', 'is_aware') 22049
('is>*', 'is_aware') 20704
('is>*', 'is_*', 'is_aware') 20704
('what>*',) 20518
('is_not',) 15977
('is_*', 'is_not') 15977
('is>*', 'is_not') 13408
('is>*', 'is_*', 'is_not') 13408
('accept_*',) 11867
('agree_is',) 11059
('agree_*', 'agree_is') 11059
('agree_does', 'agree_is') 10613


Having "trained", or fitted our model, we can then use it to annotate each (question) utterance in the corpus with the phrasings this utterance contains.

In [50]:
corpus = pm_model.transform(corpus)

10000/433787 utterances processed
20000/433787 utterances processed
30000/433787 utterances processed
40000/433787 utterances processed
50000/433787 utterances processed
60000/433787 utterances processed
70000/433787 utterances processed
80000/433787 utterances processed
90000/433787 utterances processed
100000/433787 utterances processed
110000/433787 utterances processed
120000/433787 utterances processed
130000/433787 utterances processed
140000/433787 utterances processed
150000/433787 utterances processed
160000/433787 utterances processed
170000/433787 utterances processed
180000/433787 utterances processed
190000/433787 utterances processed
200000/433787 utterances processed
210000/433787 utterances processed
220000/433787 utterances processed
230000/433787 utterances processed
240000/433787 utterances processed
250000/433787 utterances processed
260000/433787 utterances processed
270000/433787 utterances processed
280000/433787 utterances processed
290000/433787 utterances proc

One thing to note here is that each sentence can and probably will have multiple phrasings it embodies. For instance, two sentences with phrasing `agree_do` and `agree_will` will also have phrasing `agree_*`. Intuitively, more finely-specified phrasings (i.e., `agree_does`) more closely specify the phrasing embodied by a sentence (we could imagine "Do you agree..." and "Will you agree..." being very different, but perhaps also more similar to each other than "Can you explain.."). 

We want to keep track of both the complete set of phrasings and the most finely-specified phrasing you can have for each utterance. Therefore, `PhrasingMotifs` actually annotates utterances with _two_ fields.

`motifs` lists all the phrasings (arcs in a phrasing motif are separated by two underscores, `'__'`):

In [51]:
utt.get_info('motifs')

['agree_* agree_*__does>* does>*',
 'agree_* agree_*__agree_also agree_*__does>* does>*',
 'as>* share_* share_*__share_does']

and `motifs__sink` lists the most finely specified _sink phrasings_ (they are "sinks" in the sense that if you think of phrasings as a directed graph where A-->B when B is a more finely-specified version of A, these sinks have no child phrasings which are contained in the utterance)

In [52]:
utt.get_info('motifs__sink')

['agree_*__does>*', 'agree_*__agree_also', 'as>* share_*__share_does']

We'll save a subset of our output to disk -- the filtered arcs, and the motifs, potentially for use in a later transformer.

In [53]:
corpus.dump_info('utterance', ['motifs', 'motifs__sink', 'arcs_censored'])

### model persistence

We can save `pm_model` to disk and later reload it, thus caching the trained model (i.e., the motifs in a corpus and the internal representation of these motifs). Here, we save the model to a `pm_model` subfolder in the corpus directory via `dump_model()`:

In [54]:
import os

In [55]:
pm_model.dump_model(os.path.join(ROOT_DIR, 'pm_model'))

writing itemset counts
writing downlinks
writing itemset to ids
writing meta information


This subfolder then stores the motifs, as well as relations between the motifs that facilitate transforming new utterances.

In [56]:
pm_model_dir = os.path.join(ROOT_DIR, 'pm_model')
!ls $pm_model_dir

downlinks.json      itemset_counts.json itemset_to_ids.json meta.json


Suppose we later initialize a new `PhrasingMotifs` model, `new_pm_model`.

In [57]:
new_pm_model = PhrasingMotifs('motifs_new','question_arcs',min_support=100,fit_filter=question_filter,
                          verbosity=VERBOSITY)

Calling `load_model()` then reloads the stored model from our earlier run into this new model:

In [58]:
new_pm_model.load_model(os.path.join(ROOT_DIR, 'pm_model'))

reading itemset counts
reading downlinks
reading itemset to ids
reading meta information


Just to check that we've loaded the same thing that we previously saved, we'll get the motifs in our test utterance using `new_pm_model`:

In [59]:
utt = new_pm_model.transform_utterance(utt)

This is the output from the original run:

In [60]:
utt.get_info('motifs__sink')

['agree_*__does>*', 'agree_*__agree_also', 'as>* share_*__share_does']

And we see the new output matches.

In [61]:
utt.get_info('motifs_new__sink')

['agree_*__does>*', 'agree_*__agree_also', 'as>* share_*__share_does']

### example variation: not removing the nouns

**note** this takes a while to run, and is somewhat of an extension -- you can safely skip these cells.

There are other ways to use `PhrasingMotifs` that might be more or less suited to your own application. For instance, you may wonder what happens if we do not remove the nouns (as we did with `CensorNouns` above). To try this out, we can create an alternate pipeline that uses `TextToArcs` to generate root arcs (setting argument `root_only=True`) on the original parses, not the noun-censored ones.

In [62]:
q_arc_pipe_full = ConvokitPipeline([
    ('shallow_arcs_full', TextToArcs('root_arcs', input_field='parsed', 
                               root_only=True, verbosity=VERBOSITY)),
    ('question_sentence_filter', QuestionSentences('question_arcs_full', input_field='root_arcs',
                                         input_filter=question_filter, verbosity=VERBOSITY)),

])
corpus = q_arc_pipe_full.transform(corpus)


10000/433787 utterances processed
20000/433787 utterances processed
30000/433787 utterances processed
40000/433787 utterances processed
50000/433787 utterances processed
60000/433787 utterances processed
70000/433787 utterances processed
80000/433787 utterances processed
90000/433787 utterances processed
100000/433787 utterances processed
110000/433787 utterances processed
120000/433787 utterances processed
130000/433787 utterances processed
140000/433787 utterances processed
150000/433787 utterances processed
160000/433787 utterances processed
170000/433787 utterances processed
180000/433787 utterances processed
190000/433787 utterances processed
200000/433787 utterances processed
210000/433787 utterances processed
220000/433787 utterances processed
230000/433787 utterances processed
240000/433787 utterances processed
250000/433787 utterances processed
260000/433787 utterances processed
270000/433787 utterances processed
280000/433787 utterances processed
290000/433787 utterances proc

We can then train a new `PhrasingMotifs` model that finds phrasings with the nouns still included.

In [68]:
noun_pm_model = PhrasingMotifs('motifs_full','question_arcs_full',min_support=100,
                               fit_filter=question_filter, 
                          verbosity=VERBOSITY)
noun_pm_model.fit(corpus)

counting frequent itemsets for 325339 sets
	first pass: counting itemsets up to and including 5 items large
	first pass: 10000/325339 sets processed
	first pass: 20000/325339 sets processed
	first pass: 30000/325339 sets processed
	first pass: 40000/325339 sets processed
	first pass: 50000/325339 sets processed
	first pass: 60000/325339 sets processed
	first pass: 70000/325339 sets processed
	first pass: 80000/325339 sets processed
	first pass: 90000/325339 sets processed
	first pass: 100000/325339 sets processed
	first pass: 110000/325339 sets processed
	first pass: 120000/325339 sets processed
	first pass: 130000/325339 sets processed
	first pass: 140000/325339 sets processed
	first pass: 150000/325339 sets processed
	first pass: 160000/325339 sets processed
	first pass: 170000/325339 sets processed
	first pass: 180000/325339 sets processed
	first pass: 190000/325339 sets processed
	first pass: 200000/325339 sets processed
	first pass: 210000/325339 sets processed
	first pass: 220000

The most common phrasings, of course, won't be very topic-specific (unless people talk about yachts very very frequently in parliament). However, we do see that phrasings now reflect the pronoun used (which may be troublesome if we believe that "Does _he_ agree" and "Does _she_ agree" are getting at similar things).

In [69]:
noun_pm_model.print_top_phrasings(25)

('*',) 325339
('will>*',) 70226
('does>*',) 61032
('is_*',) 57964
('is>*',) 45268
('is>*', 'is_*') 42850
('agree_*',) 36108
('agree_does',) 33704
('agree_*', 'agree_does') 33704
('agree_*', 'does>*') 30012
('agree_does', 'does>*') 29987
('agree_*', 'agree_does', 'does>*') 29987
('will>the',) 26218
('will>*', 'will>the') 26218
('will>he',) 23049
('will>*', 'will>he') 23049
('is_aware',) 22063
('is_*', 'is_aware') 22063
('does>the',) 20932
('does>*', 'does>the') 20932
('what>*',) 20791
('is>*', 'is_aware') 20707
('is>*', 'is_*', 'is_aware') 20707
('does>he',) 16417
('does>*', 'does>he') 16417


Here are the sink phrasings for our example utterance from earlier, comparing against the noun-less run:

In [70]:
utt = noun_pm_model.transform_utterance(utt)

In [71]:
utt.get_info('motifs__sink')

['agree_*__does>*', 'agree_*__agree_also', 'as>* share_*__share_does']

In [72]:
utt.get_info('motifs_full__sink')

['agree_*__agree_hon',
 'agree_*__agree_also__agree_he',
 'as>* share_*__share_hon']

We see that we get this extra "hon" -- which actually stands for "honourable [member]" -- an artefact of parliamentary etiquette. 

For our particular dataset, removing nouns has the benefit of removing most of these etiquette-related words. However, you may also imagine cases where nouns actually carry a lot of useful information about rhetorical intent (including in this domain -- one could argue that asking about a person versus asking about a department is a strong signal of trying to get at different things, for instance). As such, noun-removal is something that you may want to play around with, and/or try to improve upon. 

## PromptTypes

As we intuited above, "do you agree" and "do you share my opinion" are both getting at similar intentions. However, extracting these phrasings alone won't allow us to make this association. Rather, our strategy will be to produce vector representations of them which encode this similarity. Clustering these representations then gives us different "types of question".

Our key intuition here is that questions with similar intentions will tend to be answered in similiar ways. Thus, "do you agree" and "do you share" may both often be answered with "yes, I agree"; if tomorrow I asked a new question of this ilk ("do you agree that we should invest in planes, instead of yachts"), I might be expecting a similar sort of answer. 

For a full explanation of this idea, and how we operationalized it, you can read our paper. In ConvoKit, we implement this methodology of producing vector representations and clusterings via the `PromptTypes` transformer:

In [73]:
from convokit.prompt_types import PromptTypes

`PromptTypes` will train a model -- a low-dimensional embedding, along with a k-means clustering -- by using question-answer pairs as input. 

In [74]:
def question_filter(utt, aux_input={}):
    return utt.meta['is_question']
def response_filter(utt, aux_input={}):
    return (not utt.meta['is_question']) and (utt.reply_to is not None)

We initialize `pt` with the following arguments:

* `n_types=8`: we want to infer 8 types of questions.
* `prompt_field='motifs'`: we want to encode questions in terms of the phrasing motifs we extracted above. thus, `pt` will produce representations of these motifs (rather than, e.g., the raw tokens in a question)
* `ref_field='arcs_censored'`: we will encode responses in terms of the noun-less arcs we extracted above (in practice, this appears to work better than using phrasings of responses as well, perhaps because responses are noisier)
* `prompt_filter=question_filter` and `ref_filter=response_filter`: To tell the transformer what counts as a question and an answer, we will pass the constructor the above filters (i.e., boolean functions). Note that in a less questions-heavy dataset, we could omit these filters and hence infer types of "prompts" beyond questions.
* `prompt_transform_field='motifs__sink'`: while we want to come up with a representation of _all_ phrasing motifs, when we produce a vector representation of a _particular_ utterance we want to use the most finely-specified phrasing.

There are some other arguments you can set, which are listed in the docstring. 



In [75]:
pt = PromptTypes(n_types=8, prompt_field='motifs', ref_field='arcs_censored', 
                 prompt_filter=question_filter, ref_filter=response_filter,
                 prompt_transform_field='motifs__sink',
                 output_field='prompt_types',
    random_state=1000, verbosity=1)

We can fit `pt` to the corpus -- that is, learn the associations between question phrasings and response dependency arcs that allow us to produce our vector representations, as well as a clsutering of these representations that gives us our different question types.

In [76]:
pt.fit(corpus)

fitting 195441 input pairs
fitting ref tfidf model
fitting prompt tfidf model
fitting svd model
fitting 8 prompt types


calling `display_type()` as below will print the question phrasings, response arcs, and prototypical questions and responses that are associated with each inferred type of question. We will examine some of these types more closely by way of examples, below.

In [77]:
for i in range(8):
    print(i)
    pt.display_type(i, corpus=corpus, k=15)
    print('\n\n')

0
top prompt:


,0,1,2,3,4,5,6,7,type_id
made_*,0.627821,1.112966,1.253935,1.080853,1.263088,1.081064,1.085615,1.120296,0.0
made_*__made_in,0.670131,1.092172,1.296672,1.044795,1.165740,1.096866,1.117699,1.102888,0.0
made_*__made_to,0.677337,1.226368,1.219402,1.145383,1.388485,1.110612,1.212821,1.180075,0.0
in>*__tell_*,0.681406,1.176855,0.986156,1.121633,1.332023,0.847615,0.987093,1.266240,0.0
made_*__made_what,0.683845,1.124455,1.353602,1.139765,1.248646,1.112601,1.209657,0.959844,0.0
made_*__made_been,0.689245,1.122980,1.277784,1.117208,1.263149,1.144714,1.177270,1.178932,0.0
happen_*__happen_will,0.697615,1.202465,1.101319,1.157835,1.233026,0.868512,1.052773,1.120683,0.0
made_*__what>*,0.698273,1.148946,1.360852,1.140067,1.247890,1.156193,1.226739,0.997630,0.0
made_*__made_been__made_what,0.706422,1.105542,1.336376,1.133772,1.224077,1.158618,1.230570,1.052753,0.0
made_*__made_has,0.707376,1.123585,1.303389,1.181350,1.307120,1.138856,1.224905,1.109336,0.0


top response:


,0,1,2,3,4,5,6,7,type_id
am_at,0.744773,0.935439,1.205010,1.062946,1.220152,1.025650,1.073168,1.238502,0.0
known_*,0.746823,1.227288,1.121113,1.157730,1.273498,1.041451,1.067706,1.242575,0.0
can_*,0.785983,1.107881,1.200609,1.058894,1.083504,0.993568,1.070240,0.982096,0.0
place_*,0.789185,1.049869,1.162727,1.044718,1.156990,1.036115,1.094712,1.125281,0.0
assure_have,0.796057,0.944208,1.301825,0.915244,0.966593,1.071265,1.028048,0.994812,0.0
was_made,0.797752,1.188686,0.977958,1.147366,1.179836,0.915078,0.944907,1.195879,0.0
make_shall,0.802670,0.873247,1.176298,0.898143,0.952539,0.898050,0.834410,1.129150,0.0
give_can,0.804878,1.036402,1.154003,1.068707,1.223465,0.858061,1.072920,1.131898,0.0
have_made,0.811640,1.045384,1.182388,0.922776,1.063783,1.034568,0.824843,1.211438,0.0
write_shall,0.815864,1.160921,1.133396,1.170030,1.271814,1.098569,1.200611,1.185541,0.0


top prompts:
1996-02-06a.122.5 As the House will be interested today in the change of personnel at British Gas , what direct contacts have been made with British Gas on the laying of the pipeline ? What issues were discussed ? On what dates were meetings held and what recommendations , if any , were made by the Ministry of Defence ?
['as>* made_*__made_be made_*__made_been__made_have made_*__made_been__made_what made_*__made_have__made_what made_*__made_on__made_what', 'discussed_* what>*', 'made_*__made_on on>*']

2011-05-05c.764.8 The Secretary of State made a point about less prescriptive service requirements , but will he give a guarantee that stations such as Runcorn mainline station and Widnes station in my constituency , which have seen a significant increase in passengers in the past five years , will not , as a result of his reform of franchising , have a reduction in the number of stopping trains ?
['give_*__give_will made_*']

2005-02-07a.1170.4 My understanding is that if t

...

,0,1,2,3,4,5,6,7,type_id
doing_*__what>*,1.188908,1.179227,1.296108,1.284275,1.161041,1.174012,1.380595,0.487743,7.0
doing_*,1.196526,1.177769,1.272685,1.269288,1.144117,1.162529,1.344962,0.501558,7.0
taking_*__taking_is__what>*,1.126557,1.190315,1.332255,1.244363,1.160392,1.187807,1.405707,0.508016,7.0
doing_*__doing_is__what>*,1.194910,1.201274,1.301613,1.311563,1.213732,1.199367,1.419399,0.529425,7.0
take_*__take_what,1.156225,1.012067,1.335862,1.188823,1.090698,1.169019,1.304218,0.532164,7.0
taking_*__taking_are,1.087618,1.218423,1.353556,1.237473,1.191860,1.198082,1.402136,0.533785,7.0
taking_*,1.134108,1.230914,1.339015,1.254403,1.189327,1.215009,1.418624,0.534918,7.0
will>*__work_*__work_with,1.055845,0.969658,1.376481,1.118873,1.019038,1.158587,1.261221,0.535301,7.0
taking_*__what>*,1.084641,1.223158,1.355838,1.253558,1.196290,1.203940,1.413722,0.540402,7.0
doing_*__doing_is,1.204739,1.219920,1.278352,1.299628,1.208292,1.188003,1.396126,0.541201,7.0


top response:


,0,1,2,3,4,5,6,7,type_id
through>*,1.173298,1.242311,1.328201,1.279822,1.054140,1.258116,1.369618,0.642623,7.0
is_working,1.119604,1.152739,1.275667,1.203138,0.976927,1.155934,1.216139,0.645541,7.0
ensuring_is,1.195901,1.089027,1.243815,1.110770,0.954979,1.149191,1.229360,0.648673,7.0
supporting_are,1.214342,1.245588,1.238312,1.291103,1.178745,1.182181,1.357689,0.653380,7.0
working_on,1.134486,1.224916,1.291671,1.280997,1.308231,1.218774,1.380054,0.664101,7.0
supporting_*,1.220306,1.258811,1.250189,1.318995,1.196743,1.203403,1.381304,0.666979,7.0
ensuring_*,1.175195,1.123449,1.225120,1.152106,0.926351,1.104832,1.186094,0.669527,7.0
working_are,1.136606,1.229650,1.284925,1.282427,1.311925,1.214676,1.377935,0.670170,7.0
working_with,1.136615,1.228151,1.286453,1.281580,1.314418,1.215308,1.379528,0.672401,7.0
working_*,1.138597,1.230087,1.284754,1.282966,1.312211,1.216490,1.379105,0.672730,7.0


top prompts:
1982-06-21a.18.8 Following our discussions with our European partners last October about breast food substitutes , what concerted action are we taking with our European partners to combat the problem ? What is being done about illiteracy in Third world countries ? Does the right hon Gentleman agree that that is one of the reasons why the substitutes are being abused ?
['following>* taking_*__taking_are taking_*__taking_what', 'done_*__done_is__done_what__what>* what>*__what>is', 'agree_*__agree_is__does>*']

2008-07-07d.1148.1 My constituent is seeking repayment of benefits that she was entitled to but did not receive as a result of an inaccurate assessment last autumn—undertaken without interviewing her . Inverness special payments team received her case in March , but the team tell me that , as of 27 June , they were processing claims received in October 2007 . My constituent faces a nine to 12-month delay in having her case processed . What is the Minister doing to tack

1998-06-18a.500.4 I congratulate the business mentioned by my hon Friend , which is a good example of what can be done when the barriers between business , finance and the academic and science side are broken down , and when the transfer of technology is encouraged . The Government are examining this area , believing that small and medium businesses , especially in the high - tech sector , have the ability to win for Britain . We are working closely with universities , financial institutions and businesses .
['congratulate_* encouraged_* encouraged_is encouraged_when', 'examining_* examining_are examining_believing', 'working_* working_are working_closely working_with']

2001-02-26a.578.4 We warmly commend the work that Lincoln City has been doing in that respect . I congratulate the supporters trust , which has taken over the ownership of Lincoln City football club . I am pleased that we put Supporters Direct in place nearly a year ago . It has been working hard with supporters trusts

When this trained model is used to transform a corpus, it will output several representations or features associated with each utterance.

In [78]:
utt = pt.transform_utterance(utt)

A vector representation encapsulating the utterance's rhetorical intent (in short, an embedding of the utterance based on the responses associated with questions containing its constituent phrasings):

In [79]:
utt.get_info('prompt_types__prompt_repr')

[-0.17102469270226717,
 0.030668340328553985,
 -0.14361175504486107,
 0.11035667766434655,
 -0.31492256514938993,
 -0.03234184926411349,
 -0.2228205945001759,
 -0.1280609796099674,
 0.17711977121503986,
 0.02081981426939169,
 -0.35362893625247066,
 -0.24038827321490527,
 -0.06125687797706181,
 -0.19491483622909125,
 -0.05056529746646962,
 -0.03309523304247386,
 -0.4150780974718108,
 -0.060148799466712886,
 -0.11378928189626422,
 -0.017504004346251805,
 -0.046416366723789806,
 -0.5430246645923409,
 0.13134581675629198,
 -0.085152651973838]

The distance between the vector of that utterance and the centroid of each cluster, i.e., type of question:

In [80]:
utt.get_info('prompt_types__prompt_dists.8')

[1.1430428918990674,
 0.9502324189972575,
 1.133707011765781,
 0.8403522127994857,
 0.38907971018922227,
 1.1182399779615053,
 0.763399658049645,
 1.11709159776387]

The particular type of question this utterance is, as well as how close it is to the centroid of that particular cluster (roughly, how well it fits that question type):

In [81]:
utt.get_info('prompt_types__prompt_type.8')

4.0

In [82]:
utt.get_info('prompt_types__prompt_type_dist.8')

0.38907971018922227

Here, we see that our running example is of question type 4, which is exemplified by phrasings like `does [the Minister] agree...` -- we may characterize the entire cluster as encapsulating "agreeing" questions which are perhaps asked helpfully to bolster the answerer's reputation.

In [83]:
pt.display_type(utt.get_info('prompt_types__prompt_type.8'), k=15)

top prompt:


,0,1,2,3,4,5,6,7,type_id
agree_*__agree_is,1.188158,1.063000,1.140767,0.952855,0.390807,1.232684,0.862497,1.120674,4.0
agree_*__agree_be__does>*,1.145270,1.019666,1.152760,0.885044,0.397438,1.200730,0.797764,1.133618,4.0
agree_*__agree_be,1.140607,1.022338,1.151803,0.878635,0.398083,1.191520,0.788680,1.138992,4.0
agree_*__agree_is__does>*,1.185667,1.067486,1.143729,0.954654,0.398762,1.238654,0.870336,1.114850,4.0
agree_*__agree_have,1.160346,1.060732,1.162782,0.954536,0.439447,1.232611,0.864388,1.130915,4.0
agree_*__agree_are,1.198654,1.093265,1.142069,0.940229,0.446017,1.247844,0.859772,1.162318,4.0
agree_*__agree_does__agree_have__does>*,1.145078,1.092075,1.144758,0.956562,0.454053,1.221288,0.848933,1.117233,4.0
agree_*__agree_are__agree_does__does>*,1.199917,1.099268,1.139137,0.944690,0.458240,1.253012,0.868788,1.158872,4.0
agree_*__agree_also,1.184892,1.135958,1.127462,1.026366,0.468385,1.265777,0.892345,1.176072,4.0
continue_*__will>*,1.153943,1.039539,1.207467,0.966704,0.474253,1.195661,0.962171,0.987554,4.0


top response:


,0,1,2,3,4,5,6,7,type_id
agree_certainly,1.180155,1.071745,1.201017,0.971265,0.461342,1.293082,0.952085,1.101750,4.0
agree_is,1.174426,1.071817,1.205062,0.985370,0.468024,1.291348,0.950482,1.095343,4.0
agree_however,1.171552,1.076513,1.193439,0.995855,0.468574,1.286440,0.933410,1.117073,4.0
agree_will,1.181980,1.042990,1.209447,1.001755,0.475267,1.291913,0.946098,1.121801,4.0
agree_also,1.194347,1.078747,1.193950,0.990467,0.476035,1.294584,0.947017,1.096503,4.0
agree_wholeheartedly,1.184639,1.092625,1.190386,1.018646,0.476682,1.280336,0.964095,1.089767,4.0
agree_absolutely,1.194103,1.062569,1.219094,0.982575,0.476712,1.294999,0.986945,1.067689,4.0
is_also,1.195960,1.051437,1.110080,0.986550,0.478089,1.101184,0.882075,1.002128,4.0
agree_be,1.164383,1.079607,1.202961,0.994257,0.481263,1.288859,0.944010,1.104167,4.0
agree_completely,1.186661,1.089710,1.208564,1.017079,0.481393,1.298300,0.976094,1.076271,4.0


We can transform the other utterances in the corpus as such:

In [84]:
corpus = pt.transform(corpus)

This utterance is of type 7: perhaps more information-seeking and querying for an update ("what steps is the Government taking, what are they doing to ensure", etc)

In [85]:
utt1.text

'Given what the Foreign Secretary has said about the importance of the Iran discussions on the nuclear agreement , what is he doing to ensure greater clarity about the baselines , the extent of the inspection regime and the consequences of infringement ? Given that the agreement will allow advanced centrifuge , the infringements might arrive a little earlier than anticipated .'

In [86]:
utt1.get_info('motifs__sink')

['doing_*__doing_ensure__doing_is doing_*__doing_is__doing_what given>*']

In [87]:
utt1.get_info('prompt_types__prompt_type.8')

7.0

In [88]:
pt.display_type(utt1.get_info('prompt_types__prompt_type.8'), k=15)

top prompt:


,0,1,2,3,4,5,6,7,type_id
doing_*__what>*,1.188908,1.179227,1.296108,1.284275,1.161041,1.174012,1.380595,0.487743,7.0
doing_*,1.196526,1.177769,1.272685,1.269288,1.144117,1.162529,1.344962,0.501558,7.0
taking_*__taking_is__what>*,1.126557,1.190315,1.332255,1.244363,1.160392,1.187807,1.405707,0.508016,7.0
doing_*__doing_is__what>*,1.194910,1.201274,1.301613,1.311563,1.213732,1.199367,1.419399,0.529425,7.0
take_*__take_what,1.156225,1.012067,1.335862,1.188823,1.090698,1.169019,1.304218,0.532164,7.0
taking_*__taking_are,1.087618,1.218423,1.353556,1.237473,1.191860,1.198082,1.402136,0.533785,7.0
taking_*,1.134108,1.230914,1.339015,1.254403,1.189327,1.215009,1.418624,0.534918,7.0
will>*__work_*__work_with,1.055845,0.969658,1.376481,1.118873,1.019038,1.158587,1.261221,0.535301,7.0
taking_*__what>*,1.084641,1.223158,1.355838,1.253558,1.196290,1.203940,1.413722,0.540402,7.0
doing_*__doing_is,1.204739,1.219920,1.278352,1.299628,1.208292,1.188003,1.396126,0.541201,7.0


top response:


,0,1,2,3,4,5,6,7,type_id
through>*,1.173298,1.242311,1.328201,1.279822,1.054140,1.258116,1.369618,0.642623,7.0
is_working,1.119604,1.152739,1.275667,1.203138,0.976927,1.155934,1.216139,0.645541,7.0
ensuring_is,1.195901,1.089027,1.243815,1.110770,0.954979,1.149191,1.229360,0.648673,7.0
supporting_are,1.214342,1.245588,1.238312,1.291103,1.178745,1.182181,1.357689,0.653380,7.0
working_on,1.134486,1.224916,1.291671,1.280997,1.308231,1.218774,1.380054,0.664101,7.0
supporting_*,1.220306,1.258811,1.250189,1.318995,1.196743,1.203403,1.381304,0.666979,7.0
ensuring_*,1.175195,1.123449,1.225120,1.152106,0.926351,1.104832,1.186094,0.669527,7.0
working_are,1.136606,1.229650,1.284925,1.282427,1.311925,1.214676,1.377935,0.670170,7.0
working_with,1.136615,1.228151,1.286453,1.281580,1.314418,1.215308,1.379528,0.672401,7.0
working_*,1.138597,1.230087,1.284754,1.282966,1.312211,1.216490,1.379105,0.672730,7.0


This utterance, on the other hand is a lot more aggressive -- perhaps _accusatory_ to the ends of putting the answerer on the spot ("will the secretary admit that the policy is a failure?")

In [89]:
utt2 = corpus.get_utterance('1987-03-04a.857.5')

In [90]:
utt2.text

'Will the Secretary of State stop giving us what is called in the pop record industry a remix of alibis , excuses and gimmicks ? Will he admit that the number of homes built to rent last year by local authorities was the lowest in 62 years , that the housing investment programme net of capital receipts was the lowest in real terms since HIPs were invented and that , even during the past three years the number of repair and improvement grants , which would bring some private homes back into use , have dropped by 100,000 ? Does not the right hon Gentleman understand that , if the private owner and the local authority are starved of resources , we are left with lengthy queues , homelessness and all the other scandals of poor housing that exist today ?'

In [91]:
utt2.get_info('motifs__sink')

['stop_*__stop_will__will>*',
 'admit_*__admit_will__will>*',
 'does>*__does>not does>*__understand_*']

In [92]:
utt2.get_info('prompt_types__prompt_type.8')

2.0

In [93]:
pt.display_type(utt2.get_info('prompt_types__prompt_type.8'), k=15)

top prompt:


,0,1,2,3,4,5,6,7,type_id
admit_*,1.201171,1.292763,0.570578,1.214884,1.266494,0.947761,0.940144,1.397194,2.0
why>*,1.111746,1.310107,0.574650,1.225390,1.281744,0.860374,0.900415,1.388247,2.0
admit_*__will>*,1.231918,1.313895,0.576193,1.257695,1.240869,0.998804,1.011262,1.359635,2.0
explain_*,1.081285,1.253725,0.576442,1.206437,1.269414,0.843906,0.912722,1.365633,2.0
explain_*__explain_will,1.084332,1.294334,0.591024,1.183642,1.188385,0.920798,0.869349,1.373634,2.0
is>*__is_*__is_true,1.168780,1.315689,0.596213,1.103997,1.143709,1.017692,0.846551,1.483289,2.0
is_*__why>*,1.184021,1.282762,0.601567,1.214840,1.174174,0.911120,0.845809,1.364674,2.0
justify_*,1.203301,1.322534,0.609482,1.251149,1.275439,0.976639,1.037237,1.358217,2.0
admit_*__admit_will__will>*,1.239174,1.311162,0.610798,1.289524,1.272271,0.984968,1.058717,1.349525,2.0
is_*__is_true,1.171478,1.337066,0.616361,1.159053,1.183389,1.032628,0.898227,1.491260,2.0


top response:


,0,1,2,3,4,5,6,7,type_id
wonder_*,1.177200,1.279819,0.594013,1.185855,1.109752,0.872080,0.870528,1.314760,2.0
failed_*,1.208959,1.330511,0.634805,1.256673,1.129173,0.963978,0.931793,1.338335,2.0
were_*,1.210541,1.366612,0.654166,1.198620,1.070178,1.068550,0.916717,1.350363,2.0
is_wrong,1.171259,1.387414,0.662125,1.253352,1.153526,0.974954,0.946674,1.315054,2.0
instead>*,1.178271,1.261010,0.675487,1.211907,1.236813,0.851639,0.999561,1.264442,2.0
talks_*,1.238792,1.229554,0.695368,1.236562,1.260476,0.900432,1.032589,1.339003,2.0
am_surprised,1.172448,1.232720,0.698857,1.211564,1.223285,1.007399,0.960399,1.332919,2.0
were_there,1.212256,1.391393,0.702025,1.248951,1.151671,1.088265,0.991262,1.355348,2.0
talks_about,1.231265,1.225220,0.706294,1.240260,1.297678,0.887904,1.046090,1.339956,2.0
was_*,1.178669,1.176640,0.713316,1.100452,0.869292,0.922654,0.726262,1.286168,2.0


Inspecting the other types should hopefully give you an intuition for the range of questions that tend to be asked in parliament, as well as the coherence of these types (which align fairly well with the output of the paper, even under different random seeds and small implementation tweaks). 

### a few caveats and potential modifications

One thorn in our sides might be that the model occasionally gets caught up on very generic motifs e.g., `'is>*'`, and as such, will fit many questions to the type containing `'is>*'` instead of going with a better signal; various optional parameters detailed in the documentation may provide incomplete solutions to this. Another caveat is that while this model allows us to associate together lexically-diverging phrasings (e.g., "will the Minister admit" and "does the Minister not realise" both serve to be accusatory towards the Minister), we are ultimately relying on the fact that our domain has a sufficient amount of lexical regularity (e.g., the institutional norms of how people talk in parliament) -- we might need to be cleverer when dealing with noisier settings where this regularity isn't guaranteed (like social media data). 

Finally, as a data-specific note, cluster 7 may be a result of the parser assuming that "Will the learned Gentleman please answer my question?" has "learned" as the root verb -- an artefact of parliamentary discourse we haven't handled. You may wish to play around with this by modifying how the data is preprocessed.

### model persistence

We can save our trained `pt_model` to disk for later use:

In [94]:
import os

In [95]:
pt.dump_model(os.path.join(ROOT_DIR, 'pt_model'))

dumping embedding model
dumping training embeddings
dumping type model 8


In broad strokes, what's loaded to disk is:

* TfIdf models that store the distribution of phrasings and arcs in the training data;
* SVD models that allow us to map raw phrasing/arc counts to vector representations;
* a KMeans model to cluster vector representations.

In [96]:
pt_model_dir = os.path.join(ROOT_DIR, 'pt_model')
!ls $pt_model_dir

U_prompt.npy              ref_df.8.tsv              train_prompt_vects.npy
U_ref.npy                 ref_tfidf_model.joblib    train_ref_df.8.tsv
km_model.8.joblib         svd_model.joblib          train_ref_ids.npy
prompt_df.8.tsv           train_prompt_df.8.tsv     train_ref_vects.npy
prompt_tfidf_model.joblib train_prompt_ids.npy


Initializing a new `PromptTypes` model and loading our saved model then allows us to use it again:

In [97]:
new_pt = PromptTypes(prompt_field='motifs', ref_field='arcs_censored', 
                 prompt_filter=question_filter, ref_filter=response_filter,
                 prompt_transform_field='motifs__sink',
                 output_field='prompt_types_new', prompt__tfidf_min_df=100,
                 ref__tfidf_min_df=100, 
    random_state=1000, verbosity=1)

In [98]:
new_pt.load_model(pt_model_dir)

loading embedding model
loading training embeddings
loading type model 8


In [99]:
utt = new_pt.transform_utterance(utt)

In [100]:
utt.get_info('prompt_types_new__prompt_type.8')

4.0

## examples of potential variations

### trying other numbers of prompt types:

Calling `refit_types(n)` will retrain the clustering component of the `PromptType` model to infer a different number of types. Suppose we only wanted 4 types of questions:

In [101]:
pt.refit_types(4)

fitting 4 prompt types


In [102]:
for i in range(4):
    print(i)
    pt.display_type(i, type_key=4, k=15)
    print('\n\n')

0
top prompt:


,0,1,2,3,type_id
why>*,0.601818,1.085396,1.179613,1.312615,0.0
explain_*,0.605775,1.057703,1.166116,1.279778,0.0
why>*__why>does,0.616713,1.016271,1.038717,1.310166,0.0
explain_*__explain_will,0.633035,1.080472,1.096639,1.297120,0.0
is_*__why>*,0.633817,1.107558,1.088035,1.305727,0.0
admit_*,0.647199,1.123084,1.171259,1.336647,0.0
has>*__has>not,0.649402,0.987770,0.933776,1.316079,0.0
explain_*__explain_is,0.649678,1.024644,1.100717,1.233558,0.0
explain_*__will>*,0.650367,1.077112,1.064484,1.263428,0.0
is>*__is_*__is_true,0.661020,1.113861,1.041213,1.407218,0.0


top response:


,0,1,2,3,type_id
wonder_*,0.630674,1.102998,1.044466,1.255320,0.0
is>*,0.675474,0.987292,1.008256,1.176453,0.0
failed_*,0.681234,1.168922,1.087177,1.286135,0.0
suggest_*,0.681979,0.963971,1.015263,1.236774,0.0
says_*,0.692276,1.000248,1.066520,1.175726,0.0
instead>*,0.695701,1.095288,1.174475,1.213796,0.0
remind_*,0.701194,0.949060,0.932849,1.299634,0.0
is_what,0.704836,1.072153,1.041561,1.167645,0.0
was_*,0.706220,1.054270,0.828728,1.237861,0.0
is_wrong,0.707512,1.172941,1.105634,1.273054,0.0





1
top prompt:


...

,0,1,2,3,type_id
is_be,0.983326,0.887748,0.518452,1.075414,2.0
is_reduce,1.055853,1.003526,0.519157,1.109797,2.0
be_interested,1.014612,0.916946,0.547527,1.035521,2.0
be_of,1.001621,0.822517,0.553019,1.100756,2.0
be_for,0.987128,0.886889,0.559593,1.109870,2.0
be_indeed,0.972489,0.847795,0.563212,1.096901,2.0
be_have,1.048217,0.840767,0.563700,1.011078,2.0
be_better,0.903746,0.903840,0.565646,1.119409,2.0
is_necessary,1.033036,0.975829,0.568678,1.079672,2.0
be_also,1.049832,0.860241,0.570089,1.007618,2.0





3
top prompt:


,0,1,2,3,type_id
taking_*__taking_is__what>*,1.295492,1.185034,1.230208,0.607057,3.0
will>*__work_*__work_with,1.309957,1.037493,1.070822,0.610824,3.0
doing_*__what>*,1.261882,1.205985,1.229969,0.613920,3.0
taking_*__taking_are,1.313213,1.179124,1.242876,0.614792,3.0
taking_*__what>*,1.316787,1.185156,1.254286,0.616616,3.0
done_*__done_being,1.258333,1.110325,1.144343,0.621616,3.0
do_*__do_help,1.258489,1.105808,1.196712,0.624412,3.0
doing_*,1.238402,1.199368,1.205838,0.627153,3.0
taking_*,1.307366,1.209135,1.249842,0.630915,3.0
will>*__work_*,1.271969,1.021488,0.995575,0.631464,3.0


top response:


,0,1,2,3,type_id
is_working,1.222887,1.148835,1.047432,0.692729,3.0
ensure_to,1.216149,1.045685,0.940809,0.698138,3.0
raises_*,1.289359,0.891337,1.076154,0.703652,3.0
through>*,1.304670,1.232791,1.162545,0.728547,3.0
taking_in,1.186808,1.101722,1.016710,0.731985,3.0
met_discuss,1.330599,0.964782,1.130157,0.732484,3.0
working_on,1.266305,1.197235,1.315516,0.735116,3.0
supporting_are,1.217897,1.233306,1.237249,0.738121,3.0
ensuring_*,1.182124,1.121590,1.002762,0.739539,3.0
ensuring_is,1.209579,1.122381,1.023778,0.740001,3.0


### trying other input formats

We may also experiment with different representations of the input text -- for instance, in lieu of using phrasing motifs we may instead pass questions into the model as just the raw arcs, similar to the responses. This can be modified by changing the `prompt_field` argument:

In [103]:
pt_arcs = PromptTypes(prompt_field='arcs_censored', ref_field='arcs_censored', 
                 prompt_filter=question_filter, ref_filter=response_filter,
                 prompt_transform_field='arcs_censored',
                 output_field='prompt_types_arcs', prompt__tfidf_min_df=100,
                 ref__tfidf_min_df=100, n_types=8,
    random_state=1000, verbosity=1)

In [104]:
pt_arcs.fit(corpus)

fitting 214798 input pairs
fitting ref tfidf model
fitting prompt tfidf model
fitting svd model
fitting 8 prompt types


In [105]:
for i in range(8):
    print(i)
    pt_arcs.display_type(i,  k=10)
    print('\n\n')

0
top prompt:


,0,1,2,3,4,5,6,7,type_id
be_would,0.529891,0.964818,1.226920,0.939504,0.918249,0.771973,0.884318,1.145255,0.0
be_not,0.530370,0.895000,1.250734,0.939678,1.017548,0.823587,0.879274,1.134778,0.0
asked_for,0.566036,1.002276,1.153131,1.015477,1.030582,0.796301,0.908066,1.027750,0.0
would>*,0.568386,0.897959,1.227223,0.969823,0.937112,0.737300,0.911911,1.049858,0.0
hope_*,0.568698,1.140511,1.104456,1.029072,0.761381,0.735195,0.936335,1.202869,0.0
be_*,0.572036,1.050015,1.079135,0.872280,0.767881,0.797040,0.771462,1.208420,0.0
will_*,0.575144,1.111116,1.269291,0.946087,0.945891,0.869441,1.058066,1.169810,0.0
bearing>*,0.581458,0.884393,1.251779,0.913755,0.963513,0.881427,0.835626,1.088200,0.0
take_will,0.589443,1.073734,1.161073,1.001249,0.827194,0.747144,0.967276,1.209549,0.0
in>*,0.590339,0.875245,1.181718,0.887059,0.916280,0.806805,0.761068,1.023823,0.0


top response:


,0,1,2,3,4,5,6,7,type_id
is_aware,0.567507,0.894577,1.163757,0.935506,0.944250,0.789681,0.778036,1.061885,0.0
be_however,0.568026,1.009577,1.122763,0.919661,0.871915,0.739309,0.764948,1.138676,0.0
be_may,0.578518,1.021780,1.082892,0.955665,0.879389,0.714347,0.789071,1.157237,0.0
be_possible,0.587127,0.963179,1.131451,0.988681,0.934369,0.788926,0.821702,1.181489,0.0
be_appropriate,0.596184,0.951954,1.123410,0.918310,0.931754,0.828037,0.745659,1.160878,0.0
note_*,0.603744,1.025815,1.289602,0.988940,1.001015,0.971495,1.066057,1.150926,0.0
however>*,0.606946,0.989311,1.167113,0.842493,0.870752,0.879818,0.777442,1.158154,0.0
be_indeed,0.610845,0.975784,1.095285,0.977176,0.989496,0.702595,0.786408,1.133642,0.0
be_would,0.612413,0.960792,1.121824,0.936766,0.900465,0.706843,0.731975,1.142114,0.0
realise_*,0.612891,0.827765,1.210775,0.976124,1.041444,0.820925,0.786484,0.996628,0.0





1
top prompt:


...

,0,1,2,3,4,5,6,7,type_id
said_*,1.007771,0.749372,1.146647,0.950860,1.060434,1.171081,0.535647,1.032958,6.0
said_have,1.015406,0.803270,1.131340,0.944497,1.052225,1.188140,0.545052,1.079588,6.0
said_be,0.955965,0.751642,1.155431,0.945890,1.069057,1.129259,0.549912,1.041251,6.0
said_in,1.038060,0.760210,1.123795,0.952098,1.042656,1.202765,0.550703,1.025735,6.0
said_has,0.964048,0.762851,1.152738,0.966716,1.060588,1.126601,0.551059,1.040329,6.0
said_as,1.026702,0.864824,1.096014,0.974081,0.988505,1.167021,0.557469,1.076606,6.0
said_already,1.009580,0.811801,1.131008,0.939459,1.038228,1.195170,0.559988,1.085100,6.0
said_are,1.028092,0.812897,1.123739,0.975124,1.021659,1.154675,0.561285,1.052133,6.0
said_is,0.999456,0.744576,1.142810,0.977048,1.055449,1.119307,0.564511,1.023917,6.0
having>*,0.865355,0.847961,1.085954,0.957158,0.903422,1.022245,0.576302,1.006352,6.0





7
top prompt:


,0,1,2,3,4,5,6,7,type_id
since_*,1.213373,0.984793,1.228459,1.200303,1.248047,1.184441,1.178557,0.563933,7.0
show_*,1.214324,0.884084,1.260694,1.225959,1.273321,1.153741,1.165774,0.584431,7.0
higher_*,1.144362,0.948267,1.257864,1.188304,1.241403,1.106863,1.146803,0.597594,7.0
fell_*,1.287662,0.917713,1.203311,1.185937,1.257328,1.301727,1.095045,0.602782,7.0
higher>*,1.200829,0.947739,1.221994,1.199018,1.225261,1.153603,1.162826,0.607491,7.0
risen_*,1.268897,0.970374,1.243428,1.181472,1.265682,1.240808,1.219532,0.616034,7.0
show_not,1.174909,0.877384,1.321176,1.215424,1.303690,1.140005,1.127767,0.625480,7.0
fallen_*,1.332391,1.071796,1.172275,1.219411,1.285289,1.256513,1.261160,0.633623,7.0
of>*,0.969567,0.864659,1.248692,1.170299,1.226273,0.956756,1.040890,0.640232,7.0
rising_*,1.258914,1.038468,1.173008,1.229713,1.243672,1.209732,1.190037,0.641301,7.0


top response:


,0,1,2,3,4,5,6,7,type_id
was_in,1.128448,0.854666,1.248275,1.168917,1.277705,1.065654,1.079326,0.588419,7.0
rising_*,1.222080,1.035094,1.196016,1.235545,1.292166,1.179795,1.225766,0.589343,7.0
is_higher,1.202293,0.988935,1.257062,1.253654,1.307331,1.140626,1.252858,0.598390,7.0
rising_is,1.210114,1.018229,1.210280,1.220725,1.276912,1.191285,1.212772,0.602744,7.0
rose_by,1.264710,1.042443,1.223610,1.217631,1.283751,1.230157,1.268580,0.604503,7.0
show_*,1.167111,0.909912,1.241077,1.244021,1.285517,1.061855,1.189429,0.608059,7.0
is_lower,1.238669,0.943992,1.267997,1.241343,1.320411,1.175372,1.208685,0.608498,7.0
are_now,1.285220,0.983058,1.158601,1.209898,1.306664,1.191377,1.183560,0.612259,7.0
rose_*,1.271496,1.053888,1.230944,1.216540,1.282343,1.244295,1.271104,0.612560,7.0
is_high,1.163276,0.957979,1.210000,1.133520,1.224124,1.115457,1.129165,0.613260,7.0


### going beyond root arcs

If we initialize the `TextToArcs` transformer with `root_only=False`, we will use arcs beyond those attached to the root of the dependency parse. This may produce neater output, especially in domains where utterances are less well-structured (see [this notebook](https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit/blob/master/examples/conversations-gone-awry/Conversations_Gone_Awry_Prediction.ipynb) for a demo of this on Wikipedia talk page data)

## storing vector representations

As mentioned above, `PromptTypes` produces a few vector representations of utterances. For efficiency, rather than storing these representations attached to the utterance (as values in utterance.meta), we store them in a corpus-wide matrix.

`get_vect_repr(utterance_id, matrix name)` allows us to access the representation of a particular utterance:

In [106]:
corpus.get_vect_repr(test_utt_id, 'prompt_types__prompt_repr')

array([-0.17102469,  0.03066834, -0.14361176,  0.11035668, -0.31492257,
       -0.03234185, -0.22282059, -0.12806098,  0.17711977,  0.02081981,
       -0.35362894, -0.24038827, -0.06125688, -0.19491484, -0.0505653 ,
       -0.03309523, -0.4150781 , -0.0601488 , -0.11378928, -0.017504  ,
       -0.04641637, -0.54302466,  0.13134582, -0.08515265])

To save all of these representations to disk, we can call the following:

In [107]:
corpus.dump_vector_reprs('prompt_types__prompt_repr')

This stores the representations (`vect_info.<FIELD NAME>.npy`) as a matrix, and the utterance IDs corresponding to each of the rows (`vect_info.<FIELD NAME>.keys`) -- both in the corpus directory.

In [108]:
ls $ROOT_DIR

conversations.json
corpus.json
full_pipe_models/
index.json
info.arcs_censored.jsonl
info.motifs.jsonl
info.motifs__sink.jsonl
info.parsed.jsonl
pm_model/
pt_model/
users.json
utterances.jsonl
vect_info.prompt_types__prompt_repr.keys
vect_info.prompt_types__prompt_repr.npy


These vector representations can later be re-loaded:

In [109]:
new_corpus = convokit.Corpus(ROOT_DIR)

In [110]:
new_corpus.vector_reprs.keys()

dict_keys([])

In [111]:
new_corpus.load_vector_reprs('prompt_types__prompt_repr')

In [112]:
new_corpus.get_vect_repr(test_utt_id, 'prompt_types__prompt_repr')

array([-0.17102469,  0.03066834, -0.14361176,  0.11035668, -0.31492257,
       -0.03234185, -0.22282059, -0.12806098,  0.17711977,  0.02081981,
       -0.35362894, -0.24038827, -0.06125688, -0.19491484, -0.0505653 ,
       -0.03309523, -0.4150781 , -0.0601488 , -0.11378928, -0.017504  ,
       -0.04641637, -0.54302466,  0.13134582, -0.08515265])